<h4>Integrantes</h4>
Ruiz Irisbel - 30.864.236

Sosa Andres - 30.096.794

#### Recoleccion de datos (API)

In [81]:
using JSON
using HTTP

localizacion = "Cusco"
inicio_sem = "2023-06-01"
final_sem = "2023-06-07" 

url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q=$(localizacion)&tp=24&date=$(inicio_sem)&enddate=$(final_sem)&format=json&key=790e4819ffc841c5b1810355241106"

respuesta = HTTP.request("GET", url)

if respuesta.status == 200
    data = JSON.parse(String(respuesta.body))
    println("Temperatura media de la semana $inicio_sem hasta el $final_sem de la ciudad de $localizacion (obtenidos datos correctamente)")
else
    println("Error al obtener datos de la semana del $inicio_sem al $final_sem: $(respuesta.status)")
end

temperatura_max = []
temperatura_min = []
temperatura = []

for day in data["data"]["weather"]
    temperaturas_max = day["maxtempC"]
    temperaturas_min = day["mintempC"]

    push!(temperatura_max, temperaturas_max)
    push!(temperatura_min, temperaturas_min)
end

for i in 1:length(temperatura_max)
    a = parse(Int, temperatura_max[i])
    b = parse(Int, temperatura_min[i]) 
    c = a + b
    push!(temperatura, div(c, 2))
end   

println(temperatura)

Temperatura media de la semana 2023-06-01 hasta el 2023-06-07 de la ciudad de Cusco (obtenidos datos correctamente)
Any[11, 10, 10, 10, 9, 10, 9]


### Análisis y Predicción de Datos Meteorológicos con Métodos de Interpolación

<h4>Parte 1: Interpolación de Taylor</h4>

<h4>Parte 2: Interpolación de Lagrange
</h4>

<h4>Parte 3: Interpolación de Hermite
</h4>

<h4>Parte 4: Interpolación Polinómica a Trozos
</h4>